# ARCLYTICS App Analytics  

This Jupyter Notebook is for doing some quick and dirty tests for running data analytics operations on the MongoDB and Redis data persistence. The purpose of which is to then put into a pipeline and then create endpoints for each type of query and analysis that can be done.

In [1]:
# Plotly imports

import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots

chart_studio.tools.set_credentials_file(
    username='codeninja55', 
    api_key='mLp691cLJDdKaNgJykR4'
)

chart_studio.tools.set_config_file(
    world_readable=True,
    sharing='public'
)

In [2]:
# imports
import datetime
from os import environ as env
from pymongo import MongoClient
import pandas as pd
from redis import Redis
import json 

In [19]:
conn = MongoClient(env.get('MONGO_URI'))

db_name = 'arc_dev'
collection = 'users'
db = conn[db_name]

## Live Logged in Users

In [57]:
date = datetime.datetime.utcnow().strftime('%Y-%m-%d')

pipeline = [
    {'$match': {'date': date}},
    {'$unwind': '$logged_in'},
    {
        '$project': {
            '_id': 0,
#             'date': '$date',
            'datetime': '$logged_in.datetime',
            'logged_in_users': '$logged_in.value'
        }
    }
]

res = db['celery_beat'].aggregate(pipeline)

df = pd.DataFrame(list(res))
df.dropna(inplace=True)

df['timestamp'] = pd.to_datetime(df['datetime'])
df.set_index('timestamp', inplace=True)

df

,datetime,logged_in_users
timestamp,,
2019-10-15 10:10:29.366,2019-10-15 10:10:29.366,0
2019-10-15 10:11:00.022,2019-10-15 10:11:00.022,0
2019-10-15 10:12:00.050,2019-10-15 10:12:00.050,0
2019-10-15 10:13:00.058,2019-10-15 10:13:00.058,2
2019-10-15 10:14:00.072,2019-10-15 10:14:00.072,2
2019-10-15 10:15:00.014,2019-10-15 10:15:00.014,3
2019-10-15 10:16:00.062,2019-10-15 10:16:00.062,3
2019-10-15 10:17:00.053,2019-10-15 10:17:00.053,2
2019-10-15 10:18:00.053,2019-10-15 10:18:00.053,2


In [60]:
fig = go.Figure()

trace = go.Scatter(x=df.index, y=df['logged_in_users'])

fig.add_trace(trace)

fig.update_layout(
  showlegend=False,
  title_text="Logged in Users",
#   xaxis_range=[
#     datetime.datetime(2019, 10, 4),
#     datetime.datetime(2019, 10, 5)
#   ],
  xaxis_rangeslider_visible=True
)

# fig.update_xaxes(dtick=1)
fig.update_yaxes=(tick0=0, dtick=1)

py.iplot(fig, filename='user_login_timestamps')

SyntaxError: invalid syntax (<ipython-input-60-16d46aba3794>, line 18)